Imports

In [18]:
from sys import monitoring
import h5py
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, Dataset
import numpy as np

# Division en ensembles
from sklearn.model_selection import train_test_split

# Création des DataLoaders
from torch.utils.data import Subset

#Creation du modele
import torch.nn as nn
import torch.nn.functional as F

#Entrainement du modele et scheduler
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

#evaluation du modele
from sklearn.metrics import classification_report

#monitoring avec tensorboard
from torch.utils.tensorboard import SummaryWriter

Chargement du dataset

In [19]:
dataset_simple_path = 'D:/Data/data/dog_dataset_no_aug.h5'

Classe pour charger le dataset no_aug

In [ ]:
class DogBreedDataset(Dataset):
    def __init__(self, dataset_simple_path, transform=None):
        self.file_path = dataset_simple_pat
        self.transform = transform
        
        # Charger uniquement les dimensions
        with h5py.File(self.file_path, "r") as f:
            if "images" in f:
                self.data_len = f["images"].shape[0] # type: ignore
            else:
                raise KeyError("Dataset 'images' not found in the HDF5 file.")
        
    def __len__(self):
        return self.data_len

    def __getitem__(self, idx):
        with h5py.File(self.file_path, "r") as f:
            image = f["images"][idx] / 255.0  # type: ignore # Normalisation
            label = f["labels"][idx] # type: ignore
        
        # Appliquer une transformation éventuelle
        if self.transform:
            image = self.transform(image)

        # Format PyTorch
        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)
        label = torch.tensor(label, dtype=torch.long)
        
        return image, label

# Initialisation des datasets
dataset = DogBreedDataset(dataset_simple_path)

# Division en ensembles
indices = list(range(len(dataset)))
train_indices, temp_indices = train_test_split(indices, test_size=0.3, random_state=42)
val_indices, test_indices = train_test_split(temp_indices, test_size=0.5, random_state=42)

# Création des DataLoaders
train_loader = DataLoader(Subset(dataset, train_indices), batch_size=64, shuffle=True)
val_loader = DataLoader(Subset(dataset, val_indices), batch_size=64, shuffle=False)
test_loader = DataLoader(Subset(dataset, test_indices), batch_size=64, shuffle=False)


Creation du modele

In [21]:
class DogBreedCNN(nn.Module):
    def __init__(self, num_classes=120):
        super(DogBreedCNN, self).__init__()

        # Couches de convolution
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)

        # Fully connected layers
        self.fc1 = nn.Linear(256 * 14 * 14, 512)
        self.fc2 = nn.Linear(512, num_classes)

        # Pooling et dropout
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.6)


    def forward(self, x):
        # Convolutional layers with ReLU and pooling
        x = self.pool(F.relu(self.conv1(x)))
        #print(f"Shape after conv1: {x.shape}")
        x = self.pool(F.relu(self.conv2(x)))
        #print(f"Shape after conv2: {x.shape}")
        x = self.pool(F.relu(self.conv3(x)))
        #print(f"Shape after conv3: {x.shape}")
        x = self.pool(F.relu(self.conv4(x)))
        #print(f"Shape after conv4: {x.shape}")


        # Flatten the output
        x = x.view(x.size(0), -1)
        #print(f"Shape after flattening: {x.shape}")

        
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

model = DogBreedCNN(num_classes=120)

In [22]:
images, labels = next(iter(train_loader))
outputs = model(images)
print(f"Model outputs shape: {outputs.shape}")

Model outputs shape: torch.Size([64, 120])


Entrainement no aug

In [23]:
for images, labels in train_loader:
    print(f"Images shape: {images.shape}")
    print(f"Sample images shape: {images.shape}")
    print(f"Sample labels: {labels}")
    break

Images shape: torch.Size([64, 3, 224, 224])
Sample images shape: torch.Size([64, 3, 224, 224])
Sample labels: tensor([ 57,  57, 107,  75,  42,  60,  81,  42, 116,  82,  65,  65,  95, 111,
         65,  38,  85,  24,  98,  26, 100,  41,   5,  76,  96, 108,  28,   2,
        116,  45,  30,   3,  54,  61,  85,  11, 107,  61, 107, 104,   3,  49,
         97,  62,  74, 113,  27, 106, 116,  90,  73,  76,  21,   1,  56,  32,
        117,  36,  60,  97,  62, 104, 107,  38])


In [24]:
# Configurations
learning_rate = 0.0001
num_epochs = 10

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)


scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

# Boucle d'entraînement
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    scheduler.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")
    print(f"Learning Rate: {scheduler.get_last_lr()[0]:.6f}")

Epoch [1/10], Loss: 4.7834
Learning Rate: 0.000100
Epoch [2/10], Loss: 4.7764
Learning Rate: 0.000100
Epoch [3/10], Loss: 4.7747
Learning Rate: 0.000100
Epoch [4/10], Loss: 4.7719
Learning Rate: 0.000100
Epoch [5/10], Loss: 4.7706
Learning Rate: 0.000010
Epoch [6/10], Loss: 4.7701
Learning Rate: 0.000010
Epoch [7/10], Loss: 4.7700
Learning Rate: 0.000010
Epoch [8/10], Loss: 4.7684
Learning Rate: 0.000010
Epoch [9/10], Loss: 4.7689
Learning Rate: 0.000010
Epoch [10/10], Loss: 4.7697
Learning Rate: 0.000001


Evaluation du modele

In [25]:
# Evaluation
y_true = []
y_pred = []

model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

# Rapport de classification
print(classification_report(y_true, y_pred, zero_division=1))

              precision    recall  f1-score   support

           0       1.00      0.00      0.00        38
           1       1.00      0.00      0.00         9
           2       1.00      0.00      0.00        28
           3       1.00      0.00      0.00        33
           4       1.00      0.00      0.00        24
           5       1.00      0.00      0.00        24
           6       1.00      0.00      0.00        12
           7       1.00      0.00      0.00        27
           8       1.00      0.00      0.00        30
           9       1.00      0.00      0.00        12
          10       1.00      0.00      0.00        34
          11       1.00      0.00      0.00        28
          12       1.00      0.00      0.00        11
          13       1.00      0.00      0.00        22
          14       1.00      0.00      0.00        20
          15       1.00      0.00      0.00        21
          16       1.00      0.00      0.00        19
          17       1.00    

Monitoring avec TensorBoard

In [26]:
# Initialisation de TensorBoard
writer = SummaryWriter()

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Ajouter la perte à TensorBoard
    writer.add_scalar('Loss/train', running_loss/len(train_loader), epoch)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Lancer TensorBoard
writer.close()


Epoch [1/10], Loss: 4.7682
Epoch [2/10], Loss: 4.7704
Epoch [3/10], Loss: 4.7699
Epoch [4/10], Loss: 4.7688
Epoch [5/10], Loss: 4.7688
Epoch [6/10], Loss: 4.7688
Epoch [7/10], Loss: 4.7677
Epoch [8/10], Loss: 4.7694
Epoch [9/10], Loss: 4.7685
Epoch [10/10], Loss: 4.7675
